# 内建序列类型

* Container sequences 容器序列  
  list, tuple, collections.deque 可以容纳不同类型的对象
* Flat sequences 扁平序列  
  str, bytes, bytearray, memoryview and array.array 可以容纳同一类型的对象
  
容器序列存放引用, 而扁平序列存放的多是值(bytes, str...).  
相比之下, 扁平序列更加紧凑, 内存占用会更少, 但是扁平类型被限制只能存放主要类型的值, 比如characters, bytes and numbers

* Mutable sequences 可变序列  
  list, bytearray, array.array, collections.deque and memoryview
* Immutable sequences 可变(固定)序列  
  tuple, str and bytes

# 将序列分解为单独的变量

任何序列(或可以迭代的对象)，都可以进行解包操作的.  
唯一的要求是解包的数量和序列的数量相同，否则会抛出异常(如`ValueError: not enough values to unpack` or `ValueError: too many values to unpack`).

In [1]:
name, ages, price, date = ['Tom', 18, 12.9, (2012, 12, 21)]
name, ages, price, date

('Tom', 18, 12.9, (2012, 12, 21))

In [2]:
x, y, z = (4, 5)

ValueError: not enough values to unpack (expected 3, got 2)

In [3]:
x, y = (4, 6, 8)

ValueError: too many values to unpack (expected 2)

In [4]:
a, b, *c = 'hello'
a, b, c

('h', 'e', ['l', 'l', 'o'])

# 保存最后N个元素

## 场景
在迭代或其他的处理过程中对最后几项记录做一个有限的历史记录统计

## 解决方案
`collections.deque`，deque(maxlen=N)会创建一个固定长度
的队列.
当新的记录加入队列而队列已满的时候，队列会弹出最老的记录

In [5]:
import collections
q = collections.deque(maxlen=3)

In [6]:
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [7]:
q.append(4)
q

deque([2, 3, 4])

而且，deque也可以作为一个非常简单的双界队列使用，可以在队列的两端使用.
并且append，pop操作的复杂度为O(1)，list为O(N)

In [8]:
print(q)
q.appendleft('left')
q

deque([2, 3, 4], maxlen=3)


deque(['left', 2, 3])

In [9]:
q.popleft()
q

'left'

deque([2, 3])

# 找到最大或最小的N个元素  
`heapq`模块中两个函数，nlargest()和nsmallest()

In [10]:
import heapq

In [11]:
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums))
print(heapq.nsmallest(3, nums))

[42, 37, 23]
[-4, 1, 2]


这两个函数可以支持自定义key用来处理更复杂的数据结构，类似于sort()一样

In [12]:
portfolio = [
    {'name': 'IBM', 'price': 91.1},
    {'name': 'APPLE', 'price': 543.1},
    {'name': 'FB', 'price': 21.1},
    {'name': 'HPQ', 'price': 1.1},
    {'name': 'YHOO', 'price': 12.1},
    {'name': 'ACME', 'price': 123.4}
]
# cheap
heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
# expensive
heapq.nlargest(3, portfolio, key=lambda s: s['price'])

[{'name': 'HPQ', 'price': 1.1},
 {'name': 'YHOO', 'price': 12.1},
 {'name': 'FB', 'price': 21.1}]

[{'name': 'APPLE', 'price': 543.1},
 {'name': 'ACME', 'price': 123.4},
 {'name': 'IBM', 'price': 91.1}]

对于`heapq`，还可以将`list`中的顺序转换为堆的顺序.
也就是说可以将list当做堆来使用，从而对于pop()的时间复杂度达到O(logN)

In [13]:
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
heapq.heapify(nums)
nums

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

In [14]:
heapq.heappop(nums)
nums

-4

[1, 2, 2, 23, 7, 8, 18, 23, 42, 37]

In [15]:
heapq.heappop(nums)
nums

1

[2, 2, 8, 23, 7, 37, 18, 23, 42]

In [16]:
heapq.heappush(nums, 8)
nums

[2, 2, 8, 23, 7, 37, 18, 23, 42, 8]

# 优先级队列

可以使用`heapq`来实现一个简单的优先级队列

In [17]:
import heapq

class PriorityQueue:
    
    def __init__(self):
        self._queue = []
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    
    def pop(self):
        return heapq.heappop(self._queue)[-1]

In [18]:
class Item:
    
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return 'Item({!r})'.format(self.name)

In [19]:
q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
q.pop()
q.pop()
q.pop()

Item('bar')

Item('spam')

Item('foo')

## Priority如何排序
对于我们自己实现了Item是无法排序，那么`heapq`是如何进行排序的？
关键点在`tuple`

In [20]:
Item('foo') > Item('bar')

TypeError: '>' not supported between instances of 'Item' and 'Item'

In [21]:
print('----','using tuple', '----')
(1, Item('foo')) > (0, Item('bar'))
(1, Item('foo')) < (10, Item('bar'))

---- using tuple ----


True

True

在`tuple`的排序中，默认是tuple对象的元素排序的.  
而在`PriorityQueue`中，使用了`(-priority, index, item)`来进行排序.  
使用了`index`可以保证，在`priority`相同时，index不相同，从而保证了在优先级相同时，`PriorityQueue`会以元素的插入顺序

# 在字典中将键映射到多个值上

In [22]:
d = {
    'a': [1, 2, 3],
    'b': [4, 5]
}
e = {
    'a': {1, 2, 3},
    'b': {4, 5}
}

In [23]:
d['a'], e['a']

([1, 2, 3], {1, 2, 3})

`list`: 可以保存插入顺序  
`set`: 可以去除重复

对于这样使用字典，为了方便使用，可以使用`defaultdict`  
`defaultdict`会自动初始化第一个参数

In [24]:
# 不使用defaultdict
a = {}
if a.get('a', None) is None:
    a['a'] = []
a['a'].append(1)
a

{'a': [1]}

In [25]:
# 使用defaultdict
from collections import defaultdict
b = defaultdict(list)
b['a'].append(1)
b

defaultdict(list, {'a': [1]})

# 保持有序的字典

In [26]:
from collections import OrderedDict

In [27]:
d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4
for key, value in d.items():
    print(key, value)

foo 1
bar 2
spam 3
grok 4


`OrderedDict`为了实现有序，维护了一个双向链表.  
`OrderedDict`的大小是普通字典的2倍多

# 字典相关的计算问题
对于字典的key, value进行计算, 一般使用`zip()`将字段的key, value进行反转

In [28]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 37.20,
    'HPQ': 37.20,
    'FB': 10.75
}

In [29]:
# min price
min(zip(prices.values(), prices.keys()))
max(zip(prices.values(), prices.keys()))

(10.75, 'FB')

(612.78, 'AAPL')

**注意**, `zip()`返回一个迭代器, 意味了它只能使用一次

In [30]:
prices_names = zip(prices.values(), prices.keys())

In [31]:
min(prices_names)

(10.75, 'FB')

In [32]:
# prices_names 已经消费完了, empty
max(prices_names)

ValueError: max() arg is an empty sequence

# 对切片命名

In [33]:
s = '12hello2132world'
# 不易阅读
s[2:7], s[-5:]
hello_indices = slice(2, 7)
world_indices = slice(-5, None)
s[hello_indices], s[world_indices]

('hello', 'world')

('hello', 'world')

`inidices()`来获取切片对象对于某个序列大小的索引

In [34]:
hello_indices.indices(len(s))

(2, 7, 1)

# 找出序列中出现次数最多的元素

In [35]:
from collections import Counter
words = ('look into my eyes look into my eyes '
'the eyes the eyes the eyes not around the '
'eyes don\'t look around the eyes look into '
'my eyes you\'re under').split()
words[1:5]

['into', 'my', 'eyes', 'look']

In [36]:
word_counts = Counter(words)
#top_three
word_counts.most_common(3)

[('eyes', 8), ('the', 5), ('look', 4)]

In [37]:
word_counts['eyes']

8

# itemgetter来替代lambda

In [38]:
from operator import itemgetter

rows = [
    {'fname': 'Brian', 'uid': 1003},
    {'fname': 'David', 'uid': 1002},
    {'fname': 'Big', 'uid': 1001},
    {'fname': 'John', 'uid': 1004}
]
sorted(rows, key=itemgetter('uid', 'fname'))

[{'fname': 'Big', 'uid': 1001},
 {'fname': 'David', 'uid': 1002},
 {'fname': 'Brian', 'uid': 1003},
 {'fname': 'John', 'uid': 1004}]

In [39]:
sorted(rows, key=lambda x: x['uid'])

[{'fname': 'Big', 'uid': 1001},
 {'fname': 'David', 'uid': 1002},
 {'fname': 'Brian', 'uid': 1003},
 {'fname': 'John', 'uid': 1004}]

`itemgetter`可以选择多个参数,并返回`tuple`  
对于多个字段的排序, 将会很有用.  
并且相比于`lambda`, `itemgetter`会有更好的性能.

# 根据字段将记录分组
## 使用itertools.groupby()

In [40]:
rows = [
    {'address': '5412  CLARK', 'date': '07/01/2012'},
    {'address': '5148  CLARK', 'date': '07/04/2012'},
    {'address': '5800  CLARK', 'date': '07/02/2012'},
    {'address': '2122  CLARK', 'date': '07/03/2012'},
    {'address': '5645  CLARK', 'date': '07/02/2012'},
    {'address': '1060  CLARK', 'date': '07/02/2012'},
    {'address': '4801  CLARK', 'date': '07/01/2012'},
    {'address': '1039  CLARK', 'date': '07/04/2012'}
]

如果要想根据日期分组的方式迭代数据, 可以先对`date`字段排序, 然后使用`itertools.groupby()`

In [41]:
from operator import itemgetter
from itertools import groupby

sorted_rows = sorted(rows,key=itemgetter('date'))

for date, items in groupby(sorted_rows, key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ', i)

07/01/2012
  {'address': '5412  CLARK', 'date': '07/01/2012'}
  {'address': '4801  CLARK', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800  CLARK', 'date': '07/02/2012'}
  {'address': '5645  CLARK', 'date': '07/02/2012'}
  {'address': '1060  CLARK', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122  CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148  CLARK', 'date': '07/04/2012'}
  {'address': '1039  CLARK', 'date': '07/04/2012'}


`groupby()`通过扫描序列找出拥有相同的值(可由key指定), 并将它们分组.  
`groupby()`创建了一个迭代器, 每次迭代返回一个值和一个字迭代器(sub_iterator).  
**注意**, `groupby()`只能检查连续的项, 需要先排序才能使用. 不然会出现奇怪的行为.

In [42]:
for date, items in groupby(rows, key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ', i)

07/01/2012
  {'address': '5412  CLARK', 'date': '07/01/2012'}
07/04/2012
  {'address': '5148  CLARK', 'date': '07/04/2012'}
07/02/2012
  {'address': '5800  CLARK', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122  CLARK', 'date': '07/03/2012'}
07/02/2012
  {'address': '5645  CLARK', 'date': '07/02/2012'}
  {'address': '1060  CLARK', 'date': '07/02/2012'}
07/01/2012
  {'address': '4801  CLARK', 'date': '07/01/2012'}
07/04/2012
  {'address': '1039  CLARK', 'date': '07/04/2012'}


## 更好的性能
因为使用`groupby()`必须需要先排序, 意味着`groupby()`的性能不是很多(排序意味了复杂度至少为O(NlogN)).  
而如果是手动将数据分组的话, **性能**会更好, 并且可以**多次使用**.(如果不考虑内存的话, 这种方式会额外生成一个字典)

In [43]:
from collections import defaultdict
rows_by_date = defaultdict(list)
# 手动将数据分组
for row in rows:
    rows_by_date[row['date']].append(row)

for date, items in rows_by_date.items():
    print(date)
    for i in items:
        print(' ', i)

07/01/2012
  {'address': '5412  CLARK', 'date': '07/01/2012'}
  {'address': '4801  CLARK', 'date': '07/01/2012'}
07/04/2012
  {'address': '5148  CLARK', 'date': '07/04/2012'}
  {'address': '1039  CLARK', 'date': '07/04/2012'}
07/02/2012
  {'address': '5800  CLARK', 'date': '07/02/2012'}
  {'address': '5645  CLARK', 'date': '07/02/2012'}
  {'address': '1060  CLARK', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122  CLARK', 'date': '07/03/2012'}


# 筛选序列中的元素
## 最简单的方法, (list comprehension)

In [44]:
mylist = [1, 4, -5, 10, -7, 2, 3, -1]
[n for n in mylist if n > 0]
[n for n in mylist if n < 0]

[1, 4, 10, 2, 3]

[-5, -7, -1]

list comprehension 会生成一个列表, 如果太大的原始数据, 可能会占用很多内存.  
如果在意的话, 可以使用生成器表达式, 通过迭代来获取筛选后的数据.

In [45]:
pos = (n for n in mylist if n > 0)
list(pos)

[1, 4, 10, 2, 3]

If you are not doing something with the produced list, you should not use that syntax.
对于多个iterable, list comprehension会生成笛卡尔积

## 处理复杂的情况
如果筛选过长比较复杂(或者需要异常处理), 那么把筛选过程写作函数然后使用`filter()`会更加方便

In [46]:
values = ['hello', '1', '2', '-3', 'N/A', '5']


def is_int(val):
    try:
        int(val)
        return True
    except ValueError:
        return False
    
list(filter(is_int, values))

['1', '2', '-3', '5']

**Notes:** `filter()`会返回一个迭代器, 所以需要使用`list`工厂方法来转换为`list`

## 使用布尔值来筛选 itertools.compress()


In [47]:
addresses = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GARNVILLE',
]
counts = [0, 3, 10, 4, 1, 7, 6, 1]

将`counts`大于5的address筛选出来

In [48]:
from itertools import compress
more5 = [n > 5 for n in counts]
'more5:', more5
list(compress(addresses, more5))

('more5:', [False, False, True, False, False, True, True, False])

['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']

# 声明List时, 注意不要加逗号

In [49]:
list_with_comma = [
  1, 2, 3  
],

list_without_comma = [
    1, 2, 3
]

list_with_comma, type(list_with_comma)
list_without_comma, type(list_without_comma)

(([1, 2, 3],), tuple)

([1, 2, 3], list)

# 从字典中提取子集, dictionary comprehension

In [50]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}

In [51]:
# Make a dictionary of all prices over 200
{key: value for key, value in prices.items() if value > 200}

{'AAPL': 612.78, 'IBM': 205.55}

In [52]:
# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
{key: value for key, value in prices.items() if key in tech_names}

{'AAPL': 612.78, 'HPQ': 37.2, 'IBM': 205.55}

同样也可以使用`tuple`传入`dict`的工厂方法实现

In [53]:
dict((key, value) for key, value in prices.items() if value > 200)

{'AAPL': 612.78, 'IBM': 205.55}

对比来说, 使用dictionary comprehension 的速度会更快

# 使用名称替代索引来访问序列元素  
`collections.namedtuple`, 只增加了比较小的开销来实现, 返回`tuple`的子类, 同样`namedtuple`是不可变(immutable)的序列, 生成了就不可变

In [54]:
from collections import namedtuple
Subscriber = namedtuple('Subscriber', ['addr', 'joined'])

In [55]:
sub = Subscriber('jonesy@example.com', '2012-10-19')
sub
sub[0], sub[1]
sub.addr, sub.joined

Subscriber(addr='jonesy@example.com', joined='2012-10-19')

('jonesy@example.com', '2012-10-19')

('jonesy@example.com', '2012-10-19')

使用`namedtuple`可以将索引位置解耦合, 使用名称来引用数据, 可以使代码的扩展性比较强

如果对于大型的数据结构列入字典, 那么可以使用`namedtuple`来实现
会更高效(替代字典), 但是`immutable`

# 同时对数据做转换(map)和换算(reduce)

reduction: sum(), min(), max()
在reduction之前, 做转换或筛选(map)

In [56]:
nums = [1, 2, 3, 4, 5]
sum(x * x for x in nums)

55

In [57]:
import os 
files = os.listdir('..')
if any(name.endswith('.py') for name in files):
    print('There be python!')
else:
    print('Sorry, no python files')

There be python!


In [58]:
s = ('ACME', 50, 123.45)
print(','.join(str(x) for x in s))

ACME,50,123.45


这种表达式是generator expression, 在函数中只有一个参数可以这样使用, 但是如果超过一个参数, 必须显式的加上括号区分

In [59]:
nums = [1, 2, 3, 4, 5]

In [60]:
sorted(x * x for x in nums)

[1, 4, 9, 16, 25]

In [61]:
sorted(x * x for x in nums, key=lambda x: x)

SyntaxError: Generator expression must be parenthesized if not sole argument (<ipython-input-61-74405d3d86c6>, line 1)

In [62]:
sorted((x * x for x in nums), key=lambda x: x)

[1, 4, 9, 16, 25]

# 将多个映射合并为单个映射
有多个字典时, 想在逻辑上将它们合并为一个单独的映射结构, 表现成一个单独的字典

In [63]:
a = {'x': 1, 'z': 3}
b = {'y': 2, 'z': 4}

In [64]:
from collections import ChainMap
c = ChainMap(a, b)

In [65]:
c['x']
c['y']
c['z']
c.values

1

2

3

<bound method Mapping.values of ChainMap({'x': 1, 'z': 3}, {'y': 2, 'z': 4})>